# Elevation Interpolation

In [8]:
# import libraries
import arcpy
from arcpy import env # to set working environmnet
import pandas as pd
import psycopg2
import os # file path managment

### Set up environmnet

In [9]:
# Get CWD

cwd = _dh[0] #this is a universal variable that calls the directory where the .ipynb file is located


# Link it to the QAQC.gdb and make it the workspace

arcpy.env.workspace = os.path.join(cwd, '..', '..', 'data', 'QAQC.gdb')

# allow files to be overwritten
arcpy.env.overwriteOutput = True 
#test
arcpy.env.workspace

'C:\\WINDOWS\\system32\\..\\..\\data\\QAQC.gdb'

## Query elevation data from database
- use SQL
- may need to convert raster to point data
- resample to avoid crashing the computer
--(may already be small enough)

In [10]:
# Get credentials

cred_pth = os.path.join(os.getcwd(), '..', '..', 'database', 'db_credentials.txt')

with open(cred_pth, 'r') as f:
    
    creds = f.readlines()[0].split(', ')

# set raster location variables
schema = 'postgres'
table = 'minneapolisdem'
column = 'rast'

# Connect to PostGIS Database

pg_connection_dict = dict(zip(['dbname', 'user', 'password', 'port', 'host'], creds))

conn = psycopg2.connect(**pg_connection_dict)




FileNotFoundError: [Errno 2] No such file or directory: 'C:\\WINDOWS\\system32\\..\\..\\database\\db_credentials.txt'

In [35]:
cur = conn.cursor()
cur.execute("SELECT * FROM minneapolisdem;")


In [38]:
import arcpy

geodatabase_path = r"C:\Users\MrJDF\Desktop\arc2_lab2\mpls_prplair_qaqc\mpls_prplair_qaqc.gdb"

raster = cur

arcpy.RasterToGeodatabase_conversion(raster, geodatabase_path)

# for row in cur:
#     raster = row[0]
#     #metadata1 = row[1]
#    # metadata2 = row[2]
#     arcpy.RasterToGeodatabase_conversion(raster, geodatabase_path)


RuntimeError: Object: Error in executing tool

In [27]:
# Close cursor

cur.close()

# Close connection

conn.close()

In [11]:

# Define the input raster in the Postgres database
input_raster = "PG:dbname=lab0 host=34.123.228.238 port=5432 user=postgres password=MonsterKitty3232 schema=postgres table=minneapolisdem column=rast"

# Define the output geodatabase and feature dataset
output_gdb = arcpy.env.workspace
feature_dataset = "Elevation"

# Create the output geodatabase
#arcpy.CreateFileGDB_management(output_gdb, "output.gdb")

# Import the raster to the geodatabase
arcpy.RasterToGeodatabase_conversion(input_raster, output_gdb + "/" + feature_dataset)

print("Import complete.")


ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Input Rasters: Dataset PG:dbname=lab0 host=34.123.228.238 port=5432 user=postgres password=MonsterKitty3232 schema=postgres table=minneapolisdem column=rast does not exist or is not supported
ERROR 000732: Output Geodatabase: Dataset C:\WINDOWS\system32\..\..\data\QAQC.gdb/Elevation does not exist or is not supported
Failed to execute (RasterToGeodatabase).


### Sample the dataset
- define appropriate sample size (with proof)
- use random sample to sample a random distributed representative number of points
    - did this with numerical spatial data, figure out what method we used to get the random sample and do that.

### Run 3 different interpolation algorithms
- ordinary Krieging
- IDW
- universal Krieging
- spline if time

### Save the interpolated maps to my GDB and PostGIS DB
- use raster2psql as subprocess
- save raster to gdb should be basic

## Evaluate the interpolated estimates of each model compared to the true DEM
- leave one out CROSS VALIDATION in geoprocessing arcpy
- store this accuracy assessment as a table in postgres
- and in home gdb

## Find and store differencesbetween ground truth and model predictions in a point layer
- visualize this like in Luke's slides
- save to proj GDB and postgres